# Analysis of [experiment_name]

* This is a template for analyzing data that comes from mm3. 
* I would copy this file to your experimental_folder/analysis/cell_data folder and change the name to reflect your experiment. 
* Some of the paths below assume this is the location of this notebook, but you can clearly change that.
* You will need your cells.pkl file from mm3_Segment.py or mm3_OutputData.py.
* Also need your parameters.yaml and specs file to navigate the data.
* You will also need to link to mm3_helpers.py and mm3_plots.py in you mm3 directory. 
* All of the plotting and filtering functions are actually in one of these two modules.

## Load modules

In [ ]:
from __future__ import division

# import modules
from pprint import pprint # for human readable file output
try:
    import cPickle as pickle
except:
    import pickle
import numpy as np
import scipy.stats as sps
import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format
from IPython.display import display, HTML

# you need to import mm3_helpers
%load_ext autoreload
%autoreload 2 # this will reload mm3_plots.py everytime you run something. 
sys.path.insert(0, '../../mm3/') # path to your mm3 folder. 
%aimport mm3_plots

import mm3_helpers as mm3

# plotting modules and settings. 
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 18}
mpl.rc('font', **font)
mpl.rcParams['figure.figsize'] = 10, 10
mpl.rcParams['pdf.fonttype'] = 42
import seaborn as sns
sns.set(style="ticks", color_codes=True, font_scale=1.25)

## Load experiment specific files
* Here you load your parameters, specs, and cell data

In [ ]:
param_file_path = '../../params_sal1.yaml'
params = mm3.init_mm3_helpers(param_file_path)

# load specs file
with open(params['ana_dir'] + 'specs.pkl', 'r') as specs_file:
    specs = pickle.load(specs_file)
    
# load cell data dict
with open(params['cell_dir'] + 'complete_cells.pkl', 'r') as cell_file:
    Complete_Cells = pickle.load(cell_file)

## Describe the data

In [ ]:
# how many channels do we have analyzed? 
channel_no = 0
for fov_id in specs.keys():
    for peak_id, spec in specs[fov_id].items():
        if spec == 1:
            channel_no += 1
print('There are %d channels analyzed' % channel_no)

## Plot basic stats

In [ ]:
# This creates a dataframe of key cell parameters. 
Complete_Cells_df = mm3_plots.cells2df(Complete_Cells)

# Calculate basic stats
columns = ['sb', 'sd', 'delta', 'tau', 'elong_rate', 'septum_position']
display(Complete_Cells_df[columns].describe())

## Filter cells
* mm3_plots.py and mm3_helpers.py have convinient functions for filtering.

In [ ]:
Cells = mm3_plots.find_cells_of_birth_label(Complete_Cells, label_num=2)
Cells = mm3_plots.find_cells_born_before(Cells, born_before=300)
Cells = mm3_plots.filter_by_stat(Cells, center_stat='mean', std_distance=3)

## Plot basic stats of filtered cells
* All analysis going forward will be done on the filtered cells.

In [ ]:
Cells_df = mm3_plots.cells2df(Cells)
columns = ['sb', 'sd', 'delta', 'tau', 'elong_rate', 'septum_position']
display(Cells_df[columns].describe())

## Parameters across FOVs

In [ ]:
fig, ax = mm3_plots.violin_fovs(Cells_df)

# Figure caption
txt = 'Figure caption text'
fig.text(.1, 0.025, txt)
plt.subplots_adjust(bottom=0.075, left=0.075, right=0.95)

# show and save figure
plt.show()
fig.savefig('./plots/cell_parameters_by_fov.png', dpi=100)

## Parameters vs birth_label

In [ ]:
fig, ax = mm3_plots.violin_birth_label(Cells_df)

fig.set_size_inches(7.5, 15)
txt = '''Cell parameters versus birth label. Cells with birth labels 5, 6, and 7 
(of which there were very few), were first discarded. Then, cells were filtered 
to be within 3 standard deviations of the mean. '''
fig.text(.1,0.025,txt)
plt.subplots_adjust(bottom=0.1)

plt.show()
# fig.savefig('./plots/cell_parameters_by_birth_label.png', dpi=100)

## Plot parameters over time

In [ ]:
fig, ax = mm3_plots.hex_time_plot(Cells_df, time_mark='birth_time', x_extents=[0,300])

txt = 'Figure caption text'
fig.text(.1,0.025,txt)
plt.subplots_adjust(bottom=0.1, left=0.075, right=0.95)

plt.show()
# fig.savefig('./plots/cell_parameters_over_time.png', dpi=100)

## Plot cell traces over time

In [ ]:
fig, ax = mm3_plots.plot_traces(Cells)

ax[0].set_ylim([0,10])
txt = 'Figure caption text'
fig.text(.1,0.025,txt)
plt.subplots_adjust(bottom=0.1, left=0.075, right=0.95)

plt.show()
# fig.savefig('./plots/traces.png', dpi=100)

## Single cell traces over time
* Continuous lineages will only work with the mother cells

In [ ]:
Lineages = mm3_plots.organize_cells_by_channel(Cells, specs)
Lineages = mm3_plots.find_continuous_lineages(Lineages, t1=50, t2=250)

fig, ax = mm3_plots.saw_tooth_plot(Lineages, FOVs=[1,10,20,30,40], tif_width=1700, mothers=True)

plt.show()
# fig.savefig('./plots/single_cell_traces_mothers.png', dpi=100)

## Plot distributions of parameters

In [ ]:
fig, ax = mm3_plots.plot_distributions(Cells_df)

txt = 'Figure caption text'
fig.text(.1,0.05,txt)
plt.subplots_adjust(bottom=0.13, left=0.075, right=0.95)

plt.show()
# fig.savefig('./plots/distributions.png', dpi=100)

## Plot distributions rescaled by the mean

In [ ]:
fig, ax = mm3_plots.plot_rescaled_distributions(Cells_df)

txt = 'Figure caption text'
fig.text(.1,0.05,txt)
plt.subplots_adjust(bottom=0.35)

plt.show()
# fig.savefig('./plots/rescaled_distributions.png', dpi=100)

## Plot correlations
* Note you can plot the rescaled version or not. 

In [ ]:
Cells_df_r = mm3_plots.cells2df(Cells, rescale=True)

g = mm3_plots.plot_correlations(Cells_df_r, rescale=True)

txt = 'Figure caption text'
g.fig.text(.1,0.05,txt)
plt.subplots_adjust(bottom=0.15)

plt.show()
# g.fig.savefig('./plots/correlations_rescaled.png', dpi=100)